# Preparando os dados do IPTU de 2022

Para o primeiro experimento vamos preparar os dados do IPTU de 2022, e posteriormente validar com as informações apresentadas no DashBoard

In [1]:
import vaex
import geopandas as gpd
from shapely.validation import make_valid
from difflib import SequenceMatcher
import numpy as np
import pandas as pd
# import pygeos

In [2]:
gdf_distritos = gpd.read_file('data/SIRGAS_GPKG_distrito.gpkg')

gdf_quadras = gpd.read_file('zip://data/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF/SIRGAS_SHP_quadraMDSF.shp')
gdf_quadras = gdf_quadras.set_crs(epsg=31983)
gdf_quadras = gdf_quadras[gdf_quadras.qd_tipo == 'F']
gdf_quadras = gdf_quadras.dissolve(['qd_setor', 'qd_fiscal']).reset_index()
gdf_quadras_centroid = gdf_quadras.copy()
gdf_quadras_centroid.geometry = gdf_quadras_centroid.representative_point()

In [3]:
gdf_quadras.shape, gdf_quadras_centroid.shape

((45655, 10), (45655, 10))

In [4]:
gdf_quadras_distritos = gdf_quadras_centroid.sjoin(gdf_distritos, how='left', predicate='intersects')
gdf_quadras_distritos.drop('index_right', axis=1, inplace=True)

In [5]:
gdf_quadras.shape, gdf_quadras_distritos.shape

((45655, 10), (45655, 15))

In [6]:
gdf_quadras_distritos['sq'] = gdf_quadras_distritos.qd_setor + gdf_quadras_distritos.qd_fiscal

In [7]:
gpd.GeoDataFrame(gdf_quadras_distritos[['sq', 'ds_codigo', 'ds_nome']], geometry=gdf_quadras.geometry,
                crs='EPSG:31983').to_file('data/quadras.gpkg')

In [8]:
gdf_od = gpd.read_file('zip://data/SIRGAS_SHP_origemdestino_2017.zip!SIRGAS_SHP_origemdestino_2017')
gdf_od_sp = gdf_od[gdf_od.od_municip == '36']
gdf_od_sp.set_crs(epsg=31983, inplace=True)
gdf_quadras_distritos_od = gdf_quadras_distritos.sjoin(gdf_od_sp, how='left', predicate='intersects')

df_quadras_distritos_od = gdf_quadras_distritos_od.loc[:, ['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub', 'ds_subpref', 'od_id', 'od_nome']]
df_quadras_distritos_od.loc[:, ['sq']] = df_quadras_distritos_od.qd_setor + df_quadras_distritos_od.qd_fiscal
# df_quadras_distritos_od.drop_duplicates(keep='first', inplace=True, ignore_index=True)

/home/feromes/miniconda3/envs/vaex_geo_data/lib/python3.10/site-packages/geopandas/geodataframe.py:1456: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [9]:
gdf_distritos.sort_values('ds_nome')

,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,geometry
38,1,MOOCA,25,ARA,AGUA RASA,"POLYGON ((341093.380 7391912.491, 341093.329 7..."
55,2,PINHEIROS,11,API,ALTO DE PINHEIROS,"POLYGON ((326739.457 7395585.840, 326741.473 7..."
70,3,PERUS,01,ANH,ANHANGUERA,"POLYGON ((318695.858 7403032.323, 318677.730 7..."
43,4,ARICANDUVA-FORMOSA-CARRAO,26,ARI,ARICANDUVA,"POLYGON ((344068.781 7390960.855, 344068.149 7..."
11,5,PENHA,21,AAL,ARTUR ALVIM,"POLYGON ((349416.226 7397718.808, 349416.884 7..."
...,...,...,...,...,...,...
77,90,VILA MARIANA,12,VMN,VILA MARIANA,"POLYGON ((332359.740 7389124.109, 332309.630 7..."
14,91,PENHA,21,VMT,VILA MATILDE,"POLYGON ((341949.933 7396395.900, 341934.334 7..."
10,92,VILA MARIA-VILA GUILHERME,07,VMD,VILA MEDEIROS,"POLYGON ((340579.417 7401672.553, 340579.493 7..."
47,93,VILA PRUDENTE,29,VPR,VILA PRUDENTE,"POLYGON ((341731.295 7388459.495, 341723.755 7..."


In [10]:
gdf_quadras.shape, df_quadras_distritos_od.shape

((45655, 10), (45655, 9))

In [11]:
gdf_ponderacao_censo = gpd.read_file('data/areas-ponderacao-censo.gpkg', layer='areas-ponderacao-censo-2010')
df_quadras_censo =  gdf_quadras_centroid.sjoin(gdf_ponderacao_censo, how='left', predicate='intersects')

In [12]:
df_quadras_distritos_od.shape, df_quadras_censo[['COD_AED', 'COD_AED_S']].shape

((45655, 9), (45655, 2))

In [13]:
df_quadras_distritos_od = pd.concat(
    [df_quadras_distritos_od,
    df_quadras_censo[['COD_AED', 'COD_AED_S']]],
    axis=1
)

In [14]:
df_quadras_distritos_od.columns

Index(['qd_setor', 'qd_fiscal', 'ds_codigo', 'ds_nome', 'ds_cd_sub',
       'ds_subpref', 'od_id', 'od_nome', 'sq', 'COD_AED', 'COD_AED_S'],
      dtype='object')

In [15]:
gdf_od_sp = gdf_od_sp.to_file('data/OD-sp.gpkg', driver='GPKG')

In [16]:
df_quadras_distritos_od

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sq,COD_AED,COD_AED_S
0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003,3550308005004,4
1,001,004,9,BOM RETIRO,09,SE,7,Luz,001004,3550308005004,4
2,001,005,9,BOM RETIRO,09,SE,7,Luz,001005,3550308005004,4
3,001,006,9,BOM RETIRO,09,SE,7,Luz,001006,3550308005004,4
4,001,007,9,BOM RETIRO,09,SE,7,Luz,001007,3550308005004,4
...,...,...,...,...,...,...,...,...,...,...,...
45650,310,116,38,JABAQUARA,15,JABAQUARA,274,Jabaquara,310116,3550308005095,95
45651,310,117,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310117,3550308005093,93
45652,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118,3550308005093,93
45653,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310119,3550308005093,93


In [17]:
## TODO
# Verificar essa inconsistencia
gdf_quadras_distritos[gdf_quadras_distritos.ds_codigo.isna()]

,qd_setor,qd_fiscal,geometry,qd_id_orig,qd_tipo,qd_subqua,qd_area,qd_situac,qd_id,qd_tx_tipo,ds_codigo,ds_subpref,ds_cd_sub,ds_sigla,ds_nome,sq
27467,133,127,POINT (357031.736 7402052.859),10122413,F,001,65776.7480405944,1,179963,FISCAL,NaN,NaN,NaN,NaN,NaN,133127
33243,156,135,POINT (343167.543 7387776.019),8985852,F,001,2484.92461758186,1,148338,FISCAL,NaN,NaN,NaN,NaN,NaN,156135
37127,173,160,POINT (333525.605 7378933.173),9704470,F,001,1348.82739798853,1,182464,FISCAL,NaN,NaN,NaN,NaN,NaN,173160
37132,173,165,POINT (333590.700 7378465.176),9704234,F,001,2641.57734002054,1,186397,FISCAL,NaN,NaN,NaN,NaN,NaN,173165


In [18]:
dfs = []

for i in range(1995,2023):

    print(f'Processando {i}')
    
    # df = vaex.open(f'data/IPTU_{i}/IPTU_{i}.hdf5')
    df = vaex.open(f'data//IPTU-HDF5/IPTU_{i}/IPTU_{i}.hdf5')

    df.setor = df['NUMERO DO CONTRIBUINTE'].str.slice(0,3)
    df.quadra = df['NUMERO DO CONTRIBUINTE'].str.slice(3,6)
    df.fillna(value='00-0', column_names=['NUMERO DO CONDOMINIO'], inplace=True)
    df.fillna(value=1., column_names=['FRACAO IDEAL'], inplace=True)
    df.fillna(value=0., column_names=['AREA CONSTRUIDA', 'AREA OCUPADA', 'AREA DO TERRENO'], inplace=True)
    df.sqlc = df.func.where(df['NUMERO DO CONDOMINIO'] == '00-0',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 10) + '00',
                                    df['NUMERO DO CONTRIBUINTE'].str.slice(0, 6) + '0000' + df['NUMERO DO CONDOMINIO'].str.slice(0, 2))
    df['sq'] = df.setor + df.quadra
    df['sqlc'] = df.sqlc
    df_sqlc_ac = df.groupby('sqlc', agg={'area_contruida_total': vaex.agg.sum('AREA CONSTRUIDA')})
    df = df.join(df_sqlc_ac, on='sqlc')
    df['fracao_ideal'] = df.func.where(df['FRACAO IDEAL'] == 0.,  
                                                df['AREA CONSTRUIDA'] / df['area_contruida_total'],
                                                df['FRACAO IDEAL'])
    df['ca'] = (df['AREA CONSTRUIDA'] / df['fracao_ideal']) / df['AREA DO TERRENO']
    df['to'] = df['AREA OCUPADA'] / df['AREA DO TERRENO']

    df['valor_terreno'] = df['fracao_ideal'] * df['VALOR DO M2 DO TERRENO'] * df['AREA DO TERRENO']
    df['valor_construcao'] = df['VALOR DO M2 DE CONSTRUCAO'] * df['AREA CONSTRUIDA']

    dfs.append(df)
    # break


Processando 1995
Processando 1996
Processando 1997
Processando 1998
Processando 1999
Processando 2000
Processando 2001
Processando 2002
Processando 2003
Processando 2004
Processando 2005
Processando 2006
Processando 2007
Processando 2008
Processando 2009
Processando 2010
Processando 2011
Processando 2012
Processando 2013
Processando 2014
Processando 2015
Processando 2016
Processando 2017
Processando 2018
Processando 2019
Processando 2020
Processando 2021
Processando 2022


In [19]:
df_iptu = vaex.concat(dfs)

In [20]:
df_iptu.shape

(82322059, 37)

In [24]:
columns_to_drop = [
    'NUMERO DO CONTRIBUINTE',
    'NUMERO DA NL',
    'DATA DO CADASTRAMENTO',
    'NOME DE LOGRADOURO DO IMOVEL', 
    'NUMERO DO IMOVEL', 
    'COMPLEMENTO DO IMOVEL',
    'BAIRRO DO IMOVEL', 
    'REFERENCIA DO IMOVEL',
    'CEP DO IMOVEL',
    'QUANTIDADE DE ESQUINAS FRENTES',
    'VALOR DO M2 DO TERRENO',
    'VALOR DO M2 DE CONSTRUCAO' , 
    'TIPO DE TERRENO', 
    'ANO DE INICIO DA VIDA DO CONTRIBUINTE',
    'MES DE INICIO DA VIDA DO CONTRIBUINTE', 
    'FASE DO CONTRIBUINTE', 
    'FRACAO IDEAL'
]

In [27]:
df_iptu.drop(columns=columns_to_drop, inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0


In [29]:
df_iptu.shape

(82322059, 20)

In [33]:
## Parece que para alguns lotes condominiais o terreno está proporcional à fracao ideal
df_iptu[df_iptu.ca > 60]['ca', 'AREA CONSTRUIDA', 'fracao_ideal', 'AREA DO TERRENO', 'area_contruida_total', 'ANO DO EXERCICIO']

#,ca,AREA CONSTRUIDA,fracao_ideal,AREA DO TERRENO,area_contruida_total,ANO DO EXERCICIO
0,62.370370370370374,99,0.01175771971496437,135,8420,1995
1,62.370370370370374,95,0.011282660332541567,135,8420,1995
2,62.370370370370374,106,0.012589073634204275,135,8420,1995
3,62.370370370370374,95,0.011282660332541567,135,8420,1995
4,62.370370370370374,106,0.012589073634204275,135,8420,1995
...,...,...,...,...,...,...
"1,639",62.3982652768011,23,9.999999747378752e-05,3686,31879,2022
"1,640",62.3982652768011,23,9.999999747378752e-05,3686,31879,2022
"1,641",62.3982652768011,23,9.999999747378752e-05,3686,31879,2022
"1,642",62.3982652768011,23,9.999999747378752e-05,3686,31879,2022


In [34]:
df_iptu[df_iptu['NUMERO DO CONDOMINIO'] == '00-0'].shape

(44504583, 20)

In [35]:
df_iptu.shape

(82322059, 20)

In [36]:
df_quadras_distritos_od['sqod'] = df_quadras_distritos_od.sq + df_quadras_distritos_od.od_id.str.pad(width=3, side='left', fillchar='0')

In [37]:
df_quadras_distritos_od.shape

(45655, 12)

In [38]:
df_quadras_distritos_od

,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,sq,COD_AED,COD_AED_S,sqod
0,001,003,9,BOM RETIRO,09,SE,7,Luz,001003,3550308005004,4,001003007
1,001,004,9,BOM RETIRO,09,SE,7,Luz,001004,3550308005004,4,001004007
2,001,005,9,BOM RETIRO,09,SE,7,Luz,001005,3550308005004,4,001005007
3,001,006,9,BOM RETIRO,09,SE,7,Luz,001006,3550308005004,4,001006007
4,001,007,9,BOM RETIRO,09,SE,7,Luz,001007,3550308005004,4,001007007
...,...,...,...,...,...,...,...,...,...,...,...,...
45650,310,116,38,JABAQUARA,15,JABAQUARA,274,Jabaquara,310116,3550308005095,95,310116274
45651,310,117,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310117,3550308005093,93,310117275
45652,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310118,3550308005093,93,310118275
45653,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,310119,3550308005093,93,310119275


In [39]:
df_quadras_distritos_od = df_quadras_distritos_od.drop_duplicates(subset=['sq'])

In [40]:
df_quadras_distritos_od.shape, df_iptu.shape

((45655, 12), (82322059, 20))

In [41]:
df_iptu = df_iptu.join(vaex.from_pandas(df_quadras_distritos_od), on='sq', how='left',  
            allow_duplication=False, inplace=False)

In [42]:
df_quadras_distritos_od.shape, df_iptu.shape

((45655, 12), (82322059, 31))

## Adicionando informações de Uso

In [43]:
df_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()
df_usos_2022

,TIPO DE USO DO IMOVEL,count
0,Residência e outro uso (predominância residenc...,85643
1,"Outras edificações de uso comercial, com utili...",22163
2,Posto de serviço,2279
3,"Prédio de apartamento, não em condomínio, de u...",2209
4,"Estação radioemissora, de televisão ou empresa...",495
5,Flat de uso comercial (semelhante a hotel),24619
6,Apartamento em condomínio,1439450
7,Garagem (unidade autônoma) em edifício em cond...,200254
8,Clube esportivo,573
9,"Outras edificações de uso especial, com utiliz...",1263


In [44]:
df_usos = df_iptu.groupby('TIPO DE USO DO IMOVEL').agg('count').to_pandas_df()

In [45]:
df_usos.fillna('', inplace=True)

In [46]:
results, ratios = [], []

for i, row in df_usos.iterrows():

    similaridade = df_usos_2022['TIPO DE USO DO IMOVEL'].apply(lambda x: SequenceMatcher(None, row['TIPO DE USO DO IMOVEL'], x).ratio())
    
    results.append(df_usos_2022.iloc[similaridade.idxmax()]['TIPO DE USO DO IMOVEL'])

    ratios.append(similaridade.max())

df_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [47]:
df_usos

,TIPO DE USO DO IMOVEL,count,titulo_padronizado,relacao_de_assertividade_com_titulo_padronizado
0,,26,Residência e outro uso (predominância residenc...,0.000000
1,"Outras edificações de uso comercial, com utili...",147516,"Outras edificações de uso comercial, com utili...",1.000000
2,Residência e outro uso (predominância residenc...,595522,Residência e outro uso (predominância residenc...,1.000000
3,"outras edificações do tipo (uso especial), com...",24600,"Outras edificações de uso especial, com utiliz...",0.912000
4,Garagem (unidade autônoma) em edifício em cond...,178032,Garagem (unidade autônoma) em edifício em cond...,0.895706
...,...,...,...,...
73,"asilo, orfanato, creche, seminário ou convento",11795,"Asilo, orfanato, creche, seminário ou convento",0.978261
74,cortiço (habitação coletiva subnormal),41620,Residência e outro uso (predominância residenc...,0.363636
75,escola,135829,Escola,0.833333
76,"Prédio de escritório ou consultório, não em co...",173977,"Prédio de escritório ou consultório, não em co...",1.000000


In [48]:
residencial = ['residencial', 'residência', 'apartamento', 'cortiço']
comercial = ['comércio', 'comercial', 'posto', 'loja']
servicos =  ['hotel', 'radioemissora', 'flat', 'serviço', 'clube', 'oficina', 'hospital', 'templo', 'escritórios', 'consultórios', 'escritório', 'consultório', 'asilo', 'creche', 'prédio de garagens', 'escola']
industrial = ['Indústria', 'armazéns']

In [49]:
class_residencial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(residencial)), case=False)
class_comercial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(comercial)), case=False)
class_servicos = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(servicos)), case=False)
class_industrial = df_usos.loc[:, 'titulo_padronizado'].str.contains(r'\b(?:{})\b'.format('|'.join(industrial)), case=False)

df_usos.loc[class_residencial, ['classificacao_abrangente']] = 'Residencial'
df_usos.loc[class_comercial, ['classificacao_abrangente']] = 'Comercial'
df_usos.loc[class_servicos, ['classificacao_abrangente']] = 'Serviços'
df_usos.loc[class_industrial, ['classificacao_abrangente']] = 'Industrial'

df_usos.loc[df_usos.classificacao_abrangente.isna(), ['classificacao_abrangente']] = 'Outros'

In [50]:
# df_iptu['TIPO DE USO DO IMOVEL']
df_usos = vaex.from_pandas(df_usos.loc[df_usos['TIPO DE USO DO IMOVEL'] != '', ['TIPO DE USO DO IMOVEL', 'classificacao_abrangente']])
df_iptu = df_iptu.join(df_usos, on='TIPO DE USO DO IMOVEL')

In [51]:
df_iptu[['classificacao_abrangente']]

#,classificacao_abrangente
0,Comercial
1,Comercial
2,Comercial
3,Comercial
4,Comercial
...,...
"82,322,054",Residencial
"82,322,055",Residencial
"82,322,056",Residencial
"82,322,057",Outros


In [52]:
df_iptu.shape

(82322059, 32)

In [53]:
df_padroes_usos_2022 = df_iptu[df_iptu['ANO DO EXERCICIO'] == 2022].groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos_2022.loc[df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'] == 'TERRENO', 'TIPO DE PADRAO DA CONSTRUCAO'] = 'Terreno'
df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'] = df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'].str.replace('/', ',')
df_padroes_usos_2022

,TIPO DE PADRAO DA CONSTRUCAO,count
0,Residencial horizontal - padrão A,285433
1,"Oficina,Posto de serviço,Armazém,D",21814
2,Residencial vertical - padrão E,56532
3,Residencial horizontal - padrão F,2022
4,"Barracão,Telheiro,Oficina - padrão",4067
5,Residencial horizontal - padrão B,395050
6,Residencial horizontal - padrão E,8779
7,Comercial vertical - padrão B,54602
8,Residencial horizontal - padrão D,47294
9,Terreno,103197


In [104]:
df_padroes_usos = df_iptu.groupby('TIPO DE PADRAO DA CONSTRUCAO').agg('count').to_pandas_df()
df_padroes_usos.fillna('', inplace=True)
df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'] = df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'].str.replace('/', ',')
df_padroes_usos

,TIPO DE PADRAO DA CONSTRUCAO,count
0,,26
1,Residencial horizontal - padrão A,2010356
2,"Oficina,Posto de serviço,Armazém,D",152824
3,Residencial vertical - padrão E,387531
4,residência horizontal - Padrão E,181857
5,Residencial horizontal - padrão F,13752
6,residência horizontal - Padrão A,6149461
7,residência horizontal - Padrão F,11250
8,"Barracão,Telheiro,Oficina - padrão",28833
9,Residencial horizontal - padrão B,2744823


In [105]:
results, ratios = [], []

for i, row in df_padroes_usos.iterrows():

    similaridade = df_padroes_usos_2022['TIPO DE PADRAO DA CONSTRUCAO'].apply(lambda x: SequenceMatcher(None, row['TIPO DE PADRAO DA CONSTRUCAO'], x).ratio())
    
    results.append(df_padroes_usos_2022.iloc[similaridade.idxmax()]['TIPO DE PADRAO DA CONSTRUCAO'])

    ratios.append(similaridade.max())

df_padroes_usos['titulo_padronizado'] = results # Os títulos costumam variar, portanto esse campo será referência para os agrupamentos
df_padroes_usos['relacao_de_assertividade_com_titulo_padronizado'] = ratios

In [106]:
# df_padroes_usos
df_padroes_usos['tipo_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[0] if x[0][-1] != ' ' else x[0][:-1])
df_padroes_usos['padrao_de_uso'] = df_padroes_usos['titulo_padronizado'].str.split("-").apply(lambda x: x[-1][-1] if x[-1][-3] == 'o' else '')

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'A') |
                    (df_padroes_usos.padrao_de_uso == 'B'),
                    'classe_de_uso'] = 'Baixo'

df_padroes_usos.loc[df_padroes_usos.padrao_de_uso == 'C',
                    'classe_de_uso'] = 'Médio'

df_padroes_usos.loc[(df_padroes_usos.padrao_de_uso == 'D') |
                    (df_padroes_usos.padrao_de_uso == 'E') |
                    (df_padroes_usos.padrao_de_uso == 'F'),
                    'classe_de_uso'] = 'Alto'

df_padroes_usos.fillna('', inplace=True)


In [107]:
df_padroes_usos

,TIPO DE PADRAO DA CONSTRUCAO,count,titulo_padronizado,relacao_de_assertividade_com_titulo_padronizado,tipo_de_uso,padrao_de_uso,classe_de_uso
0,,26,Residencial horizontal - padrão A,0.000000,Residencial horizontal,A,Baixo
1,Residencial horizontal - padrão A,2010356,Residencial horizontal - padrão A,1.000000,Residencial horizontal,A,Baixo
2,"Oficina,Posto de serviço,Armazém,D",152824,"Oficina,Posto de serviço,Armazém,D",1.000000,"Oficina,Posto de serviço,Armazém,D",,
3,Residencial vertical - padrão E,387531,Residencial vertical - padrão E,1.000000,Residencial vertical,E,Alto
4,residência horizontal - Padrão E,181857,Residencial horizontal - padrão E,0.892308,Residencial horizontal,E,Alto
5,Residencial horizontal - padrão F,13752,Residencial horizontal - padrão F,1.000000,Residencial horizontal,F,Alto
6,residência horizontal - Padrão A,6149461,Residencial horizontal - padrão A,0.892308,Residencial horizontal,A,Baixo
7,residência horizontal - Padrão F,11250,Residencial horizontal - padrão F,0.892308,Residencial horizontal,F,Alto
8,"Barracão,Telheiro,Oficina - padrão",28833,"Barracão,Telheiro,Oficina - padrão",1.000000,"Barracão,Telheiro,Oficina",,
9,Residencial horizontal - padrão B,2744823,Residencial horizontal - padrão B,1.000000,Residencial horizontal,B,Baixo


In [108]:
outros_tipos = df_padroes_usos.tipo_de_uso.str.contains('Residencial') | df_padroes_usos.tipo_de_uso.str.contains('Comercial') | df_padroes_usos.tipo_de_uso.str.contains('Terreno')

In [109]:
df_padroes_usos.loc[~outros_tipos, 'tipo_de_uso'] = 'Outros'

In [110]:
df_padroes_usos.rename(columns={'titulo_padronizado':'padrao_construcao'}, inplace=True)

In [111]:
df_padroes_usos[['tipo_de_uso', 'padrao_de_uso', 'classe_de_uso']]

,tipo_de_uso,padrao_de_uso,classe_de_uso
0,Residencial horizontal,A,Baixo
1,Residencial horizontal,A,Baixo
2,Outros,,
3,Residencial vertical,E,Alto
4,Residencial horizontal,E,Alto
5,Residencial horizontal,F,Alto
6,Residencial horizontal,A,Baixo
7,Residencial horizontal,F,Alto
8,Outros,,
9,Residencial horizontal,B,Baixo


In [115]:
df_padroes_usos['tipo_e_classe_de_uso'] = (df_padroes_usos['tipo_de_uso'] + ' ' + df_padroes_usos['classe_de_uso']).str.rstrip()

In [116]:
df_padroes_usos = vaex.from_pandas(df_padroes_usos.loc[df_padroes_usos['TIPO DE PADRAO DA CONSTRUCAO'] != '', ['TIPO DE PADRAO DA CONSTRUCAO',  'tipo_de_uso', 'padrao_de_uso', 'classe_de_uso', 'tipo_e_classe_de_uso']])
df_iptu = df_iptu.join(df_padroes_usos, on='TIPO DE PADRAO DA CONSTRUCAO')

ValueError: column name collision: tipo_de_uso exists in both column, and no proper suffix given

In [85]:
# df_iptu['tipo_e_classe_de_uso'] = df_iptu['tipo_de_uso'] + ' ' + df_iptu['classe_de_uso']

In [86]:
df_iptu.shape, df_iptu.column_names

((82322059, 37),
 ['__NUMERO DO CONTRIBUINTE',
  'ANO DO EXERCICIO',
  '__NUMERO DO CONDOMINIO',
  'CODLOG DO IMOVEL',
  '__FRACAO IDEAL',
  '__AREA DO TERRENO',
  '__AREA CONSTRUIDA',
  '__AREA OCUPADA',
  '__VALOR DO M2 DO TERRENO',
  '__VALOR DO M2 DE CONSTRUCAO',
  'ANO DA CONSTRUCAO CORRIGIDO',
  'QUANTIDADE DE PAVIMENTOS',
  'TESTADA PARA CALCULO',
  'TIPO DE USO DO IMOVEL',
  'TIPO DE PADRAO DA CONSTRUCAO',
  'FATOR DE OBSOLESCENCIA',
  'area_contruida_total',
  'NUMERO DO CONDOMINIO',
  '__FRACAO IDEAL_1',
  'AREA CONSTRUIDA',
  'AREA OCUPADA',
  'AREA DO TERRENO',
  'sq',
  'sqlc',
  'fracao_ideal',
  'ca',
  'to',
  'valor_terreno',
  'valor_construcao',
  'qd_setor',
  'qd_fiscal',
  'ds_codigo',
  'ds_nome',
  'ds_cd_sub',
  'ds_subpref',
  'od_id',
  'od_nome',
  'COD_AED',
  'COD_AED_S',
  'sqod',
  'classificacao_abrangente',
  'padrao_construcao',
  'tipo_de_uso',
  'padrao_de_uso',
  'classe_de_uso',
  'tipo_e_classe_de_uso'])

## Processando as totalizacoes

In [87]:
gdf_distritos['area'] = gdf_distritos.area
gdf_distritos.to_crs(epsg=4674, inplace=True)

In [88]:
df_iptu['sq'] = df_iptu.qd_setor + df_iptu.qd_fiscal

In [89]:
# df_iptu.sq

In [90]:
# df_iptu.fillna(value='0', column_names=['sq'], inplace=True)
# df_iptu['sq'] = df_iptu['sq'].astype('int')
# df_iptu.categorize('sq', inplace=True)

In [91]:
# df_iptu.head(1)
df_iptu.fillna(value='0', column_names=['ds_codigo'], inplace=True)
df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu.categorize('distrito', inplace=True)
# df_iptu.head(1)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,padrao_construcao,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,Terreno,,,Terreno,38


In [92]:
# df_iptu['distrito'] = df_iptu['ds_codigo'].astype('int')
df_iptu['ano'] = df_iptu['ANO DO EXERCICIO'].astype('int')
df_iptu.categorize('ano', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,padrao_construcao,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,Terreno,,,Terreno,38,2022


In [93]:
df_iptu.fillna(value='0', column_names=['od_id'], inplace=True)
df_iptu['od'] = df_iptu['od_id'].astype('int')
df_iptu.categorize('od', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,padrao_construcao,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,Terreno,,,Terreno,38,2022,275


In [94]:
df_iptu.shape

(82322059, 40)

In [95]:
df_iptu.fillna(value='0', column_names=['ds_cd_sub'], inplace=True)
df_iptu['subprefeitura'] = df_iptu['ds_cd_sub'].astype('int')
df_iptu.categorize('subprefeitura', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,padrao_construcao,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,Terreno,,,Terreno,38,2022,275,15


In [96]:
df_iptu.fillna(value='0', column_names=['COD_AED_S'], inplace=True)
df_iptu['censo'] = df_iptu['COD_AED_S'].astype('int')
df_iptu.categorize('censo', inplace=True)

#,ANO DO EXERCICIO,CODLOG DO IMOVEL,ANO DA CONSTRUCAO CORRIGIDO,QUANTIDADE DE PAVIMENTOS,TESTADA PARA CALCULO,TIPO DE USO DO IMOVEL,TIPO DE PADRAO DA CONSTRUCAO,FATOR DE OBSOLESCENCIA,area_contruida_total,NUMERO DO CONDOMINIO,AREA CONSTRUIDA,AREA OCUPADA,AREA DO TERRENO,sq,sqlc,fracao_ideal,ca,to,valor_terreno,valor_construcao,qd_setor,qd_fiscal,ds_codigo,ds_nome,ds_cd_sub,ds_subpref,od_id,od_nome,COD_AED,COD_AED_S,sqod,classificacao_abrangente,padrao_construcao,tipo_de_uso,padrao_de_uso,classe_de_uso,tipo_e_classe_de_uso,distrito,ano,od,subprefeitura,censo
0,1995,03812-1,1924,1,13.0,loja,comercial horizontal - Padrão B,0.0,135,00-0,135,0,136,001003,001003000100,1.0,0.9926470588235294,0.0,13600.0,17415.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
1,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão A,0.0,67,00-0,67,0,90,001003,001003000200,1.0,0.7444444444444445,0.0,9000.0,6298.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
2,1995,03812-1,1944,1,8.0,loja,comercial horizontal - Padrão A,0.0,85,00-0,85,0,105,001003,001003000300,1.0,0.8095238095238095,0.0,10500.0,7990.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
3,1995,03812-1,1944,1,6.0,loja,comercial horizontal - Padrão B,0.0,103,00-0,103,0,108,001003,001003000400,1.0,0.9537037037037037,0.0,10800.0,13287.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão B,Comercial horizontal,B,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
4,1995,03812-1,1944,1,7.0,loja,comercial horizontal - Padrão A,0.0,98,00-0,98,0,120,001003,001003000500,1.0,0.8166666666666667,0.0,12000.0,9212.0,001,003,9,BOM RETIRO,09,SE,7,Luz,3550308005004,4,001003007,Comercial,Comercial horizontal - padrão A,Comercial horizontal,A,Baixo,Comercial horizontal Baixo,9,1995,7,9,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"82,322,054",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15,93
"82,322,055",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15,93
"82,322,056",2022,02235-7,1978,5,0.0,Apartamento em condomínio,Residencial vertical - padrão B,0.2,25215,01-9,84,5312,18041,310118,310118000001,0.0032999999821186066,1.4109276421746886,0.2944404412172274,136990.7245577015,168420.0,310,118,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310118275,Residencial,Residencial vertical - padrão B,Residencial vertical,B,Baixo,Residencial vertical Baixo,38,2022,275,15,93
"82,322,057",2022,19137-0,0,0,0.0,Terreno,TERRENO,0.0,0,00-0,0,0,18500,310119,310119000100,1.0,0.0,0.0,39756500.0,0.0,310,119,38,JABAQUARA,15,JABAQUARA,275,Cidade Vargas,3550308005093,93,310119275,Outros,Terreno,Terreno,,,Terreno,38,2022,275,15,93


In [103]:
df_iptu.tipo_e_classe_de_uso == 'Terreno '

Expression = str_equals(tipo_e_classe_de_uso, 'Terreno ')
Length: 82,322,059 dtype: bool (expression)
-------------------------------------------
       0  False
       1  False
       2  False
       3  False
       4  False
      ...      
82322054  False
82322055  False
82322056  False
82322057   True
82322058   True

In [100]:
(df_padroes_usos['tipo_de_uso'] + ' ' + df_padroes_usos['classe_de_uso']).unique()

['Comercial horizontal Alto',
 'Outros Alto',
 'Comercial vertical Alto',
 'Residencial vertical Alto',
 'Residencial horizontal Baixo',
 'Residencial horizontal Alto',
 'Outros ',
 'Comercial horizontal Baixo',
 'Terreno ',
 'Comercial vertical Baixo',
 'Comercial vertical Médio',
 'Residencial vertical Baixo',
 'Outros Baixo',
 'Residencial horizontal Médio',
 'Comercial horizontal Médio',
 'Residencial vertical Médio']

In [63]:
# df_iptu_grouped_distrito = \
def agrupamento(df, campo):
    return df.groupby([campo, 'ano'],agg={
                        'Quantidade de Unidades': 'count', 
                        'Quantidade de Unidades Condominiais': vaex.agg.count('sqlc',
                                                                        selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho Médio da Unidade Condominial': vaex.agg.mean('AREA CONSTRUIDA', 
                                                                        selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
                        'Tamanho médio dos Terrenos': vaex.agg.mean('AREA DO TERRENO'),
                        'Área Total dos terrenos-lotes': vaex.agg.sum('AREA DO TERRENO'),
                        'Área Total Ocupada': vaex.agg.sum('AREA OCUPADA'),
                        'Área Total Construída':vaex.agg.sum('AREA CONSTRUIDA'),
                        'Valor Total dos Terrenos': vaex.agg.sum('valor_terreno'),
                        'Valor Total das Construções': vaex.agg.sum('valor_construcao'),
                        'CA médio': vaex.agg.mean('ca'),
                        'TO médio': vaex.agg.mean('to'),
                        'CA médio em lotes condominiais': vaex.agg.mean('ca',
                                                                        selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
                        'TO médio em lotes condominiais': vaex.agg.mean('to',
                                                                        selection=df['NUMERO DO CONDOMINIO'] != '00-0'),
                        'CA médio em lotes não condominiais': vaex.agg.mean('ca',
                                                                        selection=df['NUMERO DO CONDOMINIO'] == '00-0'),
                        'TO médio em lotes não condominiais': vaex.agg.mean('to',
                                                                        selection=df['NUMERO DO CONDOMINIO'] == '00-0'),
                        'Comprimento Médio da Testada': vaex.agg.mean('TESTADA PARA CALCULO'),
                        'Número médio de Pavimentos': vaex.agg.mean('QUANTIDADE DE PAVIMENTOS'),
                        'Fator de obsolecência médio': vaex.agg.mean('FATOR DE OBSOLESCENCIA'),
                        'Residencial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['classificacao_abrangente'] == 'Residencial'),
                        'Comercial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['classificacao_abrangente'] == 'Comercial'),
                        'Serviços': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['classificacao_abrangente'] == 'Serviços'),
                        'Industrial': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['classificacao_abrangente'] == 'Industrial'),
                        'Outros': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['classificacao_abrangente'] == 'Outros'),
                        'Padrão de Construção A - Baixo': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'A'),
                        'Padrão de Construção B - Baixo': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'B'),                                                                                                                                                                
                        'Padrão de Construção C - Médio': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'C'),
                        'Padrão de Construção D - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'D'),
                        'Padrão de Construção E - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'E'),
                        'Padrão de Construção F - Alto': vaex.agg.sum('AREA CONSTRUIDA', 
                                                                    selection=df['padrao_de_uso'] == 'F'),                                                                                                                                                                                                            
    })

In [64]:
campos = ['distrito', 'subprefeitura', 'od', 'censo', 'sq']

In [65]:
dfs = []

for campo in campos:
    print(campo)
    df = agrupamento(df_iptu, campo)
    for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
        df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
    df.export_hdf5(f'data/IPTU-1995-2022-agrupados-por-{campo}.hdf5')
    dfs.append(df)

distrito


/home/feromes/miniconda3/envs/vaex_geo_data/lib/python3.10/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


subprefeitura
od
censo
sq


: 

: 

In [58]:
# for i, dist in gdf_distritos.iterrows():
#     print(dist.ds_codigo, dist.ds_nome.lower().replace(' ', '-'))
#     df_iptu_distrito = df_iptu[df_iptu.distrito == int(dist.ds_codigo)]
#     df = agrupamento(df_iptu_distrito, 'sqlc')
#     for u in ['Residencial', 'Comercial', 'Serviços', 'Industrial', 'Outros']:
#         df[f'Percentual de Uso {u}'] = df[u] / df['Área Total Construída']
#     df.export_hdf5(f"data/por_distritos/IPTU-1995-2022-agrupados-por-sqlc-{dist.ds_codigo}-{dist.ds_nome.lower().replace(' ', '-')}.hdf5")
#     # break    

63 pirituba


/home/fernando/miniconda3/envs/dash/lib/python3.9/site-packages/vaex/arrow/numpy_dispatch.py:72: RuntimeWarning: invalid value encountered in true_divide
  result_data = op['op'](a_data, b_data)


95 sao-domingos
42 jaragua
11 brasilandia
29 freguesia-do-o
21 casa-verde
13 cachoeirinha
50 limao
86 vila-guilherme
89 vila-maria
92 vila-medeiros
5 artur-alvim
59 penha
18 cangaiba
91 vila-matilde
64 ponte-rasa
28 ermelino-matarazzo
84 vila-curuca
36 itaim-paulista
31 guaianases
96 lajeado
6 barra-funda
60 perdizes
88 vila-leopoldina
40 jaguara
48 lapa
41 jaguare
66 republica
69 santa-cecilia
78 se
7 bela-vista
9 bom-retiro
14 cambuci
26 consolacao
49 liberdade
53 mooca
56 pari
80 tatuape
1 agua-rasa
8 belem
10 bras
20 carrao
85 vila-formosa
4 aricanduva
73 sao-mateus
75 sao-rafael
33 iguatemi
93 vila-prudente
72 sao-lucas
54 morumbi
67 rio-pequeno
94 vila-sonia
12 butanta
65 raposo-tavares
62 pinheiros
2 alto-de-pinheiros
35 itaim-bibi
45 jardim-paulista
17 campo-limpo
19 capao-redondo
83 vila-andrade
43 jardim-angela
46 jardim-sao-luis
79 socorro
23 cidade-dutra
30 grajau
52 marsilac
55 parelheiros
25 cidade-tiradentes
61 perus
3 anhanguera
76 sapopemba
68 sacoma
27 cursino
34 ipir

In [59]:
df

#,sqlc,ano,Quantidade de Unidades,Quantidade de Unidades Condominiais,Tamanho Médio da Unidade Condominial,Tamanho médio dos Terrenos,Área Total dos terrenos-lotes,Área Total Ocupada,Área Total Construída,Valor Total dos Terrenos,Valor Total das Construções,CA médio,TO médio,CA médio em lotes condominiais,TO médio em lotes condominiais,CA médio em lotes não condominiais,TO médio em lotes não condominiais,Comprimento Médio da Testada,Número médio de Pavimentos,Fator de obsolecência médio,Residencial,Comercial,Serviços,Industrial,Outros,Percentual de Uso Residencial,Percentual de Uso Comercial,Percentual de Uso Serviços,Percentual de Uso Industrial,Percentual de Uso Outros
0,114019001100,1995,1,0,nan,250.0,250,125,125,2750.0,16875.0,0.5,0.5,nan,nan,0.5,0.5,10.0,1.0,1.0,0,0,125,0,0,0.0,0.0,1.0,0.0,0.0
1,114019001100,1996,1,0,nan,250.0,250,125,125,6000.0,21750.0,0.5,0.5,nan,nan,0.5,0.5,10.0,1.0,1.0,0,0,125,0,0,0.0,0.0,1.0,0.0,0.0
2,114019001100,1997,1,0,nan,250.0,250,125,125,6500.0,24000.0,0.5,0.5,nan,nan,0.5,0.5,10.0,1.0,1.0,0,0,125,0,0,0.0,0.0,1.0,0.0,0.0
3,114019001100,1998,1,0,nan,250.0,250,125,125,7000.0,25250.0,0.5,0.5,nan,nan,0.5,0.5,10.0,1.0,1.0,0,0,125,0,0,0.0,0.0,1.0,0.0,0.0
4,114019001100,1999,1,0,nan,250.0,250,125,125,7000.0,25625.0,0.5,0.5,nan,nan,0.5,0.5,10.0,1.0,1.0,0,0,125,0,0,0.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"781,363",140413001800,2020,1,0,nan,1000.0,1000,500,1000,504000.0,906000.0,1.0,0.5,nan,nan,1.0,0.5,10.0,2.0,1.0,1000,0,0,0,0,1.0,0.0,0.0,0.0,0.0
"781,364",140413001800,2021,1,0,nan,1000.0,1000,500,1000,504000.0,906000.0,1.0,0.5,nan,nan,1.0,0.5,10.0,2.0,0.7599999904632568,1000,0,0,0,0,1.0,0.0,0.0,0.0,0.0
"781,365",140413001800,2022,1,0,nan,1000.0,1000,500,1000,1237000.0,1383000.0,1.0,0.5,nan,nan,1.0,0.5,10.0,2.0,0.75,1000,0,0,0,0,1.0,0.0,0.0,0.0,0.0
"781,366",142113005800,2021,1,0,nan,100.0,100,42,83,50100.0,75198.0,0.83,0.42,nan,nan,0.83,0.42,3.319999933242798,2.0,0.9800000190734863,83,0,0,0,0,1.0,0.0,0.0,0.0,0.0


In [ ]:
# df_iptu_distrito = vaex.open('data/IPTU-1995-2022-agrupados-por-distrito.hdf5')
# df_iptu_subprefeitura = vaex.open('data/IPTU-1995-2022-agrupados-por-subprefeitura.hdf5')
# df_iptu_od = vaex.open('data/IPTU-1995-2022-agrupados-por-od.hdf5')

df_iptu_distrito, df_iptu_subprefeitura, df_iptu_od, df_iptu_censo, df_iptu_sq, df_iptu_sqlc = dfs

In [ ]:
f"{format(df_iptu_distrito['Quantidade de Unidades'].sum(), ',d').replace(',', '.')} seila"

In [ ]:
df_iptu_sqlc.shape

In [ ]:
df_iptu_subprefeitura["CA médio"].max()

In [ ]:
df_iptu_od["CA médio"].max()

In [ ]:
df_iptu.sqlc